# Język Python - Wykład 3

## Closures (Domknięcia)

#### Domknięcie - funkcja wraz z otoczeniem referencyjnym (wszystkie zmienne z czasu utworzenia) - dostępne nawet po opuszczeniu otoczenia leksykalnego

Funkcje zagnieżdzone:

In [ ]:
def outer():
    x = 137
    def inner():
        print(x)
    inner()

In [ ]:
outer()

Funkcje jako obiekty

In [ ]:
def add(x,y):
    return x+y
def apply(f,x,y):
    return(f(x,y))

In [ ]:
apply(add,2,2)

Domknięcia

In [ ]:
def generate_power_func(n):
    print("id(n): %X" % id(n))
    def nth_power(x):
        return x**n
    print("id(nth_power): %X" % id(nth_power))
    return nth_power

In [ ]:
raised_to_4 = generate_power_func(4)

In [ ]:
repr(raised_to_4)

In [ ]:
del generate_power_func

In [ ]:
generate_power_func(1)

In [ ]:
raised_to_4(2)

In [ ]:
raised_to_4.__closure__[0].cell_contents

Domkniecia c.d.

In [ ]:
def outer():
    x = 137
    def inner():
        nonlocal x
        print("inner",x)
        x = 0
    print("outer pre",x)
    inner()
    print("outer post",x)

In [ ]:
outer()

### Domknięcie w Groovy

    def localMethod() {
      def localVariable = new java.util.Date()
      return { println localVariable }
    }
    def clos = localMethod()


### Domknięcie w JavaScript

    function createMarker(point, number) {
        var marker = new GMarker(point);
        var message = ["This","is","the","secret","message"];
        marker.value = number;
        GEvent.addListener(marker, "click", function() {
            var myHtml = "<b>#" + number + "</b><br/>" +
                message[number -1];
            map.openInfoWindowHtml(point, myHtml);
        });
        return marker;
    }


## Dekoratory

In [ ]:
def enhanced(function):
    def tmp(y):
        print("But now, I am enhanced")
        return function(y)
    return tmp


In [ ]:
def f(n):
    print("Just a simple method printing %d." % n)

In [ ]:
f(1)

In [ ]:
f = enhanced(f)

In [ ]:
f(1)

In [ ]:
@enhanced
def g(n):
    print("Another simple method printing %d." % n)

In [ ]:
g(2)

In [ ]:
def veryenhanced(function):
    def new(y):
        print("But now, I am very enhanced")
        return function(y)
    return new

In [ ]:
@veryenhanced
@enhanced
def h(n):
    print("So simple method printing %d." % n)

In [ ]:
h(3)

    #Example ...
    @synchronized
    @logging
    def myfunc(arg1, arg2, ...):
    # ...do something


Ciąg dalszy nastąpi za chwilę...

## Obiektowość

In [ ]:
class MyClass:
    
    def f(self,m):
        self.n = m
        return 'hello world'
    
    def __init__(self):
        self.n = 12345  #pole zwykłe
    i = 12345  #pole statyczne

m = MyClass()
m.f(1)
    

In [ ]:
print(MyClass.i)

In [ ]:
m2 = MyClass()
print(m2.i)

In [ ]:
#podstawienie
m.i -= 1
print(m.i, MyClass.i, m2.i)

In [ ]:
class MyClass(object):
    
    def method1(self, x):
        print(x)
    
    @staticmethod
    def somestaticmethod(x):
        """
        metoda statyczna nie przyjmuje argumentu 'self'- 
        nie ma dostępu do atrybutów klasy/instancji.
        """
        print(x)
            
m = MyClass()
MyClass.somestaticmethod(1)
print()
m.somestaticmethod(2)

f = MyClass.somestaticmethod
f(4)
print(f.__class__)

In [ ]:
class MyClass(object):
    
    def method1(self, x):
        print(x)
    
    @classmethod
    def someclassmethod(cls, x):
        """
        w metodzie klasy, do atrybutu __self__ funkcji,
        przypisywana jest klasa a nie instancja (jak w przypadku tradycyjnych metod)
        """
        print(cls)
        print(x)

        
m = MyClass()
MyClass.someclassmethod(1)
print
m.someclassmethod(2)

g = MyClass.someclassmethod
print(g.__class__)

In [ ]:
class Employee:
    pass


john = Employee()  # Create an empty employee
# Fill the fields of the record
john.name = 'John Doe'
john.dept = 'computer lab'
john.salary = 1000

### Nie ma pól i metod prywatnych, ale...

### Pola zaczynające się od „__” mają nazwę zmienioną

In [ ]:
class MyClass:
    
    def __init__(self):
        self.__n = 12345  #pole "prywatne"

m = MyClass()
print(dir(m))
print(m._MyClass__n)

## Metody Specjalne (wprowadzenie)

    __init__ - konstruktor
    __del__ - destruktor (nie korzystamy - bo garbage collector jest nieprzewidywalny)
    __add__, __sub__, __mul__ etc. - przeciążanie operatorów
    __lt__ (<), __gt__ (<=), __eq__ (==), __ne__ (!=,<>), etc ... porównanie
    __call__ - obiekt staje się funktorem - można go wywołać jak funkcję

In [ ]:
class MyClass(object):
    """ To jest docstring """
    
    def __init__(self):
        self.n=1
        
    def __getattr__(self, name):
        print("Looking for %s" % name)
        return 0
  

In [ ]:
a = MyClass()

print(a.n)

print()

print(a.m)

In [ ]:
print(MyClass.__dict__)

print()

print(a.__dict__)

In [ ]:
class Borg:

    __shared_state = {}

    def __init__(self):
        self.__dict__ = self.__shared_state
        
a = Borg()
b = Borg()

print(a is b)
print()

a.n = 17
print(b.n)

In [ ]:
class MyClass(object):
    """ To jest doctring """
    
    def __init__(self):
        self.n = 1
        
    def __getattribute__(self, name):
        print("Looking for %s" % name)
        return 0
      
    def __setattr__(self, name, value):
        print("Setting %s at %s" % (name, value)) #ta funkcja kłamie, to pole wcale nie ma takiej wartości
        return 0
    

In [ ]:
a = MyClass()

print(a.n)
print(a.m)

print()

a.n = 3
a.m = 2

In [ ]:
class C(object):
    def __init__(self):
        self._x = 3

    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

In [ ]:
c = C()

print(c.x)

## Dekoratory, c.d.

In [ ]:
def entryExit(f):
    def new_f():
        print("Entering", f.__name__)
        f()
        print("Exited", f.__name__)
    return new_f


In [ ]:
@entryExit
def func1():
    print("inside func1()")
func1()

In [ ]:
class entryExit(object):
    
    def __init__(self, f):
        self.f = f
        self.n = 0
    
    def __call__(self):
        self.n += 1
        print("Entering", self.f.__name__, self.n, "time" + ("s" if self.n > 1 else ""))
        self.f()
        print("Exited", self.f.__name__)

        
@entryExit
def func1():
    print("inside func1()")


In [ ]:
type(func1)

In [ ]:
func1()

In [ ]:
class Decorator(object):
    
    def __init__(self, arg):
        self.arg = arg
    
    def __call__(self, cls):
        class Wrapped(cls):
            classattr = self.arg
            def new_method(self, value):
                return value * 2
        return Wrapped

In [ ]:
@Decorator("decorated class")
class TestClass(object):
    
    def new_method(self, value):
        return value * 3
    
t = TestClass()

print(t.new_method(5))

In [ ]:
t.classattr